In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
print(f'\tOPENAI_API_KEY={os.getenv("OPENAI_API_KEY")[:20]}')

	OPENAI_API_KEY=sk-proj-iKU13YeoxNgF


# Audio Extraction

In [ ]:
# ffmpeg 는 CLI 에서 동작함다.

# >ffmpeg -i mp4파일명 -vn mp3파일명 -y

# -i 옵션 : input. 입력 파일 지정 (여러개 지정 가능)
# -vn 옵션 : disable video. 비디오는 무시하고 오디오만 추출
# -y 옵션 : 덮어쓰기 할때 y/n 사용자 입력 대디하지 않고 yes 로 진행


In [2]:
!ffmpeg -i D:\NLP2507\dataset\podcast.mp4 -vn D:\NLP2507\dataset\audio.mp3 -y

ffmpeg version 2025-12-22-git-c50e5c7778-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 15.2.0 (Rev8, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-cairo --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-liboapv --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom

## 오디오 추출함수

In [3]:
import subprocess

In [4]:
base_path = r'D:\NLP2507\dataset'  # 동영상 경로
out_path = r'D:\NLP2507\dataset\out' # 분할 오디오 추출 경로

import os
if not os.path.exists(out_path):
    os.makedirs(out_path)

video_file = 'podcast.mp4'
audio_file = 'audio.mp3'

src_path = os.path.join(base_path, video_file)
dst_path = os.path.join(base_path, audio_file)

print(src_path)
print(dst_path)


D:\NLP2507\dataset\podcast.mp4
D:\NLP2507\dataset\audio.mp3


In [6]:
def extract_audio_from_video(video_path, audio_path):
    # CLI 에서 실행할 command 준비
    # 커맨드창 (CLI) 에서 명령은 아래와 같다.
    #  > ffmpeg -i D:\dataset\podcast.mp4 -vn D:\NLP2501\dataset\audio.mp3 -y
    command = ["ffmpeg",  "-i",  video_path,  "-vn",  audio_path, "-y"]
    subprocess.run(command)

In [7]:
extract_audio_from_video(src_path, dst_path)

# Cutting the Audio

In [ ]:
# pydub 패키지
#  공식: https://github.com/jiaaro/pydub 
#  pip install pydub  <- 설치 필요  (사전에 ffmpeg 가 설치되고 경로 설정도 되어 있어야 한다)


In [8]:
from pydub import AudioSegment

In [10]:
track = AudioSegment.from_mp3(dst_path)

In [12]:
# track

In [13]:
track.duration_seconds

4422.426122448979

In [14]:
len(track)  # ms

4422426

In [16]:
# 오디오의 첫 5분을 선택
five_minutes = 5 * 60 * 1000 # 단위: ms
first_five = track[:five_minutes]

# first_five

In [17]:
first_five.duration_seconds

300.0

In [19]:
# export
first_five.export(os.path.join(out_path, 'first_five.mp3'), format='mp3')

<_io.BufferedRandom name='D:\\NLP2507\\dataset\\out\\first_five.mp3'>

In [20]:
ten_minutes = 10 * 60 * 1000

In [22]:
import math
chunks = math.ceil(len(track) / ten_minutes)

chunks  # 분할될 오디오 덩어리 개수.

8

In [23]:
# 각 오디오 덩어리들의 시작과 종료 시간을 계산
for i in range(chunks):
    start_time = i * ten_minutes
    end_time = (i + 1) * ten_minutes
    print(f"{i} start:{start_time}  end {end_time}")

0 start:0  end 600000
1 start:600000  end 1200000
2 start:1200000  end 1800000
3 start:1800000  end 2400000
4 start:2400000  end 3000000
5 start:3000000  end 3600000
6 start:3600000  end 4200000
7 start:4200000  end 4800000


In [24]:
for i in range(chunks):
    start_time = i * ten_minutes
    end_time = (i + 1) * ten_minutes

    chunk = track[start_time:end_time]
    chunk.export(os.path.join(out_path, f'chunk_{i}.mp3'), format='mp3')

# Whisper Transcript

In [25]:
import openai

In [26]:
transcript = openai.audio.transcriptions.create(
    model="whisper-1",
    file=open(os.path.join(out_path, 'chunk_0.mp3'), 'rb'),
    language='en',
)

transcript 

Transcription(text="If success is this lagging indicator of commitment now, how can you be sure that you are paying your dues? The best-selling author and host. The number one health and wellness podcast. On Purpose with Jay Shetty. Society has gone in the direction of becoming addicted to pleasure. Yes. Or pleasure-seeking. Where, from the Stoic's perspective, why did we even ever go down that road? Like, why did we leave wisdom and self-control? Or did we never have it at all and we've always been trying to balance it? Yeah. I mean, I guess that's the big question is like, why do we take something that we like too far? Yeah. Right. So the Epicureans would say like, look, drinking is great, but if you have a hangover the next day, was it actually so great? And so, you know, if you, if you push the pleasure too far, it becomes not pleasurable, but in the moment that feels very far away, right? Like in the moment you want the thing now, obviously sex is this thing for people, it's like 

In [28]:
# first_five

In [30]:
transcript.text[:300]

'If success is this lagging indicator of commitment now, how can you be sure that you are paying your dues? The best-selling author and host. The number one health and wellness podcast. On Purpose with Jay Shetty. Society has gone in the direction of becoming addicted to pleasure. Yes. Or pleasure-se'

In [31]:
import glob

def transcribe_chunks(chunk_folder, destination):
    files = glob.glob(os.path.join(chunk_folder, "chunk*.mp3"))
    files.sort()
    for file in files:
        with open(file, "rb") as audio_file, open(destination, "a") as text_file:  # append mode 로 
            print(file, '녹취록 가져오는중...', end='')
            # 각 chunk 별로 녹취록 작성.
            transcript = openai.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file,
                language="en"
            )
            
            text_file.write(transcript.text)  # 곧바로 텍스트 파일에 저장
            print('완료')

transcribe_chunks(out_path, os.path.join(out_path, 'transcript.txt'))

D:\NLP2507\dataset\out\chunk_0.mp3 녹취록 가져오는중...완료
D:\NLP2507\dataset\out\chunk_1.mp3 녹취록 가져오는중...완료


In [ ]:
# glob.glob() 은 파일 이름 순서대로 list 를 리턴한다는 보장이 없다!!